In [1]:
import collections
import copy
import cspUtil
import dataUtil

In [2]:
def countSequences(inputList, N, gap=0):
    out = collections.defaultdict(float)
    if gap == 0:
        for i in range(len(inputList)-N+1):
            if [inputList[i+k] != None for k in range(N)] == [True for _ in range(N)]:
                seq = tuple([inputList[i+k] for k in range(N)])
                out[seq] += 1
        #print out
    else:
        assert N == 2
        for i in range(len(inputList)-N+1-gap):
            if (inputList[i] != None) and (inputList[i+gap+1] != None):
                seq = tuple([inputList[i], inputList[i+gap+1]])
                out[seq] += 1
        

    factor= 1.0 / sum(out.itervalues())
    for k in out:
          out[k] *= 10*factor

    return out

In [3]:
def createRythmCSP(windowedPrimTrack, window, bi=True, tri=False):
    cspRythm = cspUtil.CSP()
    (start, end) = window
    # Creating "units" variables
    transitions = [note[1] for note in windowedPrimTrack]
    for unitNum in range(start, end+1):
        unit = windowedPrimTrack[unitNum]
        cspRythm.add_variable(('U', unitNum), [(unit[0],True),(unit[0],False)])

    if bi:
        # Create binary factors proportional to frequence of apparition of pairs
        biSequences = countSequences(transitions, 2)
        for varNum in range(start, end):
            cspRythm.add_binary_factor(('U', varNum), ('U', varNum+1), lambda u1, u2 : biSequences[(u1[1], u2[1])])
        ###### Add corner cases
        cspRythm.add_unary_factor(('U', start), lambda u : biSequences[(windowedPrimTrack[start-1][1], u[1])])
        cspRythm.add_unary_factor(('U', end)  , lambda u : biSequences[(u[1], windowedPrimTrack[end+1][1])])
     
        
        # Create binary factors with gap
        for gap in range(1,len(window)):
            biSequencesGap = countSequences(transitions, 2, gap)
            for varNum in range(start, end-gap):
                cspRythm.add_binary_factor(('U', varNum), ('U', varNum+1+gap), lambda u1, u2 : biSequencesGap[(u1[1], u2[1])])
            ###### Add corner cases
            for varNum in range(start, start+gap):
                cspRythm.add_unary_factor(('U', varNum), lambda u : biSequencesGap[(windowedPrimTrack[varNum-1-gap][1], u[1])])
            for varNum in range(end-gap, end): 
                cspRythm.add_unary_factor(('U', varNum)  , lambda u : biSequencesGap[(u[1], windowedPrimTrack[varNum+2+gap][1])])

                
        """ 
    if tri:
        # Creating "binary" variables for 3-sequences handling
        domain = [(False, False), (False, True), (True, False), (True, True)]
        for unitNum in range(start, end+1):
            cspRythm.add_variable(('B', unitNum), copy.deepcopy(domain))

        # Creating binary factors proportional to frequence of apparition of triads
        triSequences = countSequences(transitions, 3)

        for varNum in range(start, end+1):
            # Add constraint B_i[0] = U_i[1]
            cspRythm.add_binary_factor(('U', varNum), ('B', varNum), lambda u, b: u[1] == b[0])

            if varNum != end:
                # Add consitency contraints B_i[0] = B_{i+1}[1] and frequency factor
                cspRythm.add_binary_factor(('B', varNum), ('B', varNum+1), \
                                           lambda b1, b2: (b1[0] == b2[1]) * triSequences[(b1[1], b1[0], b2[0])])
        # Add corner cases
        cspRythm.add_unary_factor(('B', start), lambda b: b[1] == windowedPrimTrack[start-1])
        cspRythm.add_unary_factor(('B', end)  , \
                lambda b: triSequences[(b[1], b[0], windowedPrimTrack[end+1])] \
                        * triSequences[(b[0], windowedPrimTrack[end+1], windowedPrimTrack[end+2])])
        """
    return cspRythm

In [4]:
def createPitchCSP(windowedPitchPrimTrack, weight_uni, weight_bi, window, bi=True, tri=False):
    (start, end) = window
    cspPitch = cspUtil.CSP()
    pitchTrack = [note[0] for note in windowedPitchPrimTrack]
    # Creating "units" variables #
    # Domains
    minPitch, maxPitch = pitchTrack[0], pitchTrack[0]
    for pitch in pitchTrack:
        if type(pitch) == int:
            minPitch, maxPitch = min(minPitch, pitch), max(maxPitch, pitch)
    domains = tuple(list(range(minPitch, maxPitch+1)) + ['silence'])
    
    for pitchNum in range(start, end+1):
        cspPitch.add_variable(('U', pitchNum), copy.deepcopy(domains))

     #frequency of single apparition
    freq1 = countSequences(pitchTrack, 1)
    for pitchIdx in range(start, end):
        cspPitch.add_unary_factor(('U', pitchIdx), lambda p : freq1[(p,)]**weight_uni)
    
    if bi:
        # Create binary factor: must keep the same pitch if previous duration = True
        for varNum in range(start, end): 
            if windowedPitchPrimTrack[varNum][1]:
                cspPitch.add_binary_factor(('U', varNum), ('U', varNum+1), lambda x, y : x == y)
        ###### Add corner cases
        if windowedPitchPrimTrack[start-1][1]:
            cspPitch.add_unary_factor(('U', start), lambda x : x == windowedPitchPrimTrack[start-1][0])
        if windowedPitchPrimTrack[end][1]:
            cspPitch.add_unary_factor(('U', end), lambda x : x == windowedPitchPrimTrack[end+1][0])

        # Create binary factors proportional to frequence of apparition of pairs
        biSequences = countSequences(pitchTrack, 2)
        for varNum in range(start, end):
            cspPitch.add_binary_factor(('U', varNum), ('U', varNum+1), lambda p1, p2 : biSequences[(p1, p2)]**weight_bi)
        ###### Add corner cases
        cspPitch.add_unary_factor(('U', start), lambda p : biSequences[(pitchTrack[start-1], p)]**weight_bi)
        cspPitch.add_unary_factor(('U', end)  , lambda p : biSequences[(p, pitchTrack[end+1])]**weight_bi)

        # Create binary factors with gap
        for gap in range(1,len(window)):
            biSequencesGap = countSequences(pitchTrack, 2, gap)
            for varNum in range(start, end-gap):
                cspPitch.add_binary_factor(('U', varNum), ('U', varNum+1+gap), lambda p1, p2 : biSequencesGap[(p1, p2)]**weight_bi)
            ###### Add corner cases
            for varNum in range(start, start+gap):
                cspPitch.add_unary_factor(('U', varNum), lambda p : biSequencesGap[(pitchTrack[varNum-1-gap], p)]**weight_bi)
            for varNum in range(end-gap, end): 
                cspPitch.add_unary_factor(('U', varNum)  , lambda p : biSequencesGap[(p, pitchTrack[varNum+2+gap])]**weight_bi)


        """
    if tri:
        # Creating "binary" variables for 3-sequences handling
        domain = [(False, False), (False, True), (True, False), (True, True)]
        for unitNum in range(start, end+1):
            cspPitch.add_variable(('B', unitNum), copy.deepcopy(domain))

        # Creating binary factors proportional to frequence of apparition of triads
        triSequences = countSequences(windowedPrimTrack, 3)

        for varNum in range(start, end+1):
            # Add constraint B_i[0] = U_i[1]
            cspPitch.add_binary_factor(('U', varNum), ('B', varNum), lambda u, b: u[1] == b[0])

            if varNum != end:
                # Add consitency contraints B_i[0] = B_{i+1}[1] and frequency factor
                cspPitch.add_binary_factor(('B', varNum), ('B', varNum+1), \
                                           lambda b1, b2: (b1[0] == b2[1]) * triSequences[(b1[1], b1[0], b2[0])])
        # Add corner cases
        cspPitch.add_unary_factor(('B', start), lambda b: b[1] == windowedPrimTrack[start-1])
        cspPitch.add_unary_factor(('B', end)  , \
                lambda b: triSequences[(b[1], b[0], windowedPrimTrack[end+1])] \
                        * triSequences[(b[0], windowedPrimTrack[end+1], windowedPrimTrack[end+2])])
        """

    return cspPitch

In [42]:
def reconstruction(mypattern,  window, weight_uni, weight_bi, trackNum=0, bi=True, tri=False, rhythmRec=False):
    rythmUnit, windowedPrimTrack = mypattern.getCorrupt(window, trackNum)
    windowedPitchPrimTrack = windowedPrimTrack[:]
    if rhythmRec:
        # Rhythm reconstruction
        CSP_rythm = createRythmCSP(windowedPrimTrack, window, bi, tri)
        search_rythm = cspUtil.BacktrackingSearch()
        search_rythm.solve(CSP_rythm, mcv = False, ac3 = False)
        print search_rythm.optimalAssignment
        for i, note in enumerate(windowedPitchPrimTrack[:]):
            if note == (None, None):
                windowedPitchPrimTrack[i] = search_rythm.optimalAssignment[('U', i)]
    else:
        # No Rhythm reconstruction (rhythm based on the original score)
        for i, note in enumerate(windowedPitchPrimTrack[:]):
            if note == (None, None):
                windowedPitchPrimTrack[i] = (None, mypattern.primTracks[trackNum][i][1])

    # Pitch reconstruction
    CSP_pitch = createPitchCSP(windowedPitchPrimTrack, weight_uni, weight_bi, window, bi=True, tri=False)
    search_pitch = cspUtil.BacktrackingSearch()
    search_pitch.solve(CSP_pitch, mcv = True, ac3 = True)
    primReconstruction = windowedPitchPrimTrack[:]
    for i, note in enumerate(primReconstruction[:]):
        if note[0] == None:
            if ('U', i) in search_pitch.optimalAssignment.keys():
                primReconstruction[i] = (search_pitch.optimalAssignment[('U', i)], primReconstruction[i][1])
    
    return primReconstruction

In [43]:
def lossFunction(window, primOriginal, primReconstruction):
    (start, end) = window
    accuracy = 0
    for i in range(start, end+1):
        if primOriginal[i][0] == primReconstruction[i][0]:
            accuracy += 1
        else: 
            print 'Position:', i, 'Original pitch:', primOriginal[i][0], 'Reconstructed pitch:', primReconstruction[i][0]
            
        if primOriginal[i][1] == primReconstruction[i][1]:
            accuracy += 1
        else: 
            print 'Position:', i, 'Original rhythm:', primOriginal[i][1], 'Reconstructed rhythm:', primReconstruction[i][1]
            
    loss = 100 * (1 - accuracy/(2.*(end+1-start)))
    #print 'Loss value (percentage):', loss
    return  loss

In [44]:
midi_file = "/Users/robin/Desktop/CS221_projet/Open_MIDI_file/ode.mid"
mypattern = dataUtil.MusicPattern(midi_file)
mypattern.midiToLisp()
mypattern.lispToPrim()
mypattern.primToLisp()
mypattern.lispToMidi()

Loaded /Users/robin/Desktop/CS221_projet/Open_MIDI_file/ode.mid


In [47]:
trackNum = 0
weight_uni = 1
weight_bi = 1
window = (35, 45)
primReconstruction = reconstruction(mypattern,  window, weight_uni, weight_bi, trackNum=0, bi=True, tri=False, rhythmRec=True)
primOriginal = mypattern.primTracks[trackNum]
Loss = lossFunction(window, primOriginal, primReconstruction)
Loss

{('U', 45): (None, False), ('U', 44): (None, True), ('U', 35): (None, False), ('U', 43): (None, False), ('U', 42): (None, True), ('U', 39): (None, False), ('U', 41): (None, False), ('U', 38): (None, True), ('U', 40): (None, True), ('U', 37): (None, False), ('U', 36): (None, True)}
Position: 36 Original pitch: 77 Reconstructed pitch: 76
Position: 37 Original pitch: 77 Reconstructed pitch: 76
Position: 38 Original pitch: 79 Reconstructed pitch: 76
Position: 39 Original pitch: 79 Reconstructed pitch: 76
Position: 40 Original pitch: 79 Reconstructed pitch: 76
Position: 41 Original pitch: 79 Reconstructed pitch: 76
Position: 42 Original pitch: 77 Reconstructed pitch: 76
Position: 43 Original pitch: 77 Reconstructed pitch: 76


36.36363636363637

# Finding optimal weights for unary and binary factors

In [ ]:
trackNum = 0
weight_uni = 1
weight_bi = 1
#window = (20, 30)
for i in range(10,30):
    window = (i, i+10)
    primReconstruction = reconstruction(mypattern,  window, weight_uni, weight_bi, trackNum=0, bi=True, tri=False, rhythmRec=False)
    primOriginal = mypattern.primTracks[trackNum]
    Loss = lossFunction(window, primOriginal, primReconstruction)
    print Loss

In [ ]:
Loss_mean = []
for i in range(len(Weight_factors)):
    weight_uni = Weight_factors[i]
    for j in range(len(Weight_factors)):
        weight_bi = Weight_factors[j]
        Losses = [trainExamples[k][1] for k in range(len(trainExamples)) if (trainExamples[k][0][1] == weight_uni) & (trainExamples[k][0][2] == weight_bi)]
        Loss_mean.append([weight_uni, weight_bi, sum(Losses)/len(Losses)])

In [ ]:
min([Loss_mean[i][2] for i in range(len(Loss_mean))])

In [ ]:
[Loss_mean[i][2] for i in range(len(Loss_mean))]